# Utilização de LLMs

In [1]:
import os
import warnings
import torch

from huggingface_hub import login

print(f"Device name: '{torch.cuda.get_device_name()}'")
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7974MB, multi_processor_count=34)'
Não suporta bfloat16.


In [2]:
warnings.filterwarnings('ignore')
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "microsoft/Phi-3-mini-4k-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 12.9 s, sys: 3.13 s, total: 16.1 s
Wall time: 4.71 s


In [13]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Quem foi Einstein?"

messages = [
    {"role": "user", "content": "Olá"},
    {"role": "assistent", "content": "Tudo bem? Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

CPU times: user 11.2 s, sys: 67.1 ms, total: 11.2 s
Wall time: 11.2 s


'Albert Einstein foi um físico teórico alemão mais famoso por desenvolver a teoria da relatividade, uma das bases para a física moderna. Nascido em Ulm, Alemanha, em 14 de janeiro de 1879, Einstein passou a maior parte de sua vida profissional como cientista de ponta.\n\nEle ganhou a maioria dos prêmios e reconhecimentos da física, incluindo o Nobel de Física, em 1921, por seu contínuo trabalho visando a compreensão das leis da natureza, e, mais especificamente, pela formulação da teoria da relatividade especial. Além disso, seu artigo "A teoria da Relatividade", publicado em 1905, fundamentou a relação entre massa e energia através da famosa equação E=mc², que é uma das equações mais reconhecidas do mundo.\n\nAlém de suas contribuições a física, Einstein também se envolveu em políticas públicas e em organizações não governamentais, como o pacifismo e o pacifismo nuclear. Morreu em Princeton, Nova Jersey, em 18 de abril de 1955.'